# Analisis del equilibrio

## Importación de librerías y lectura de datos

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
df_x = pd.read_csv("../resultados/dinamica/velocidad_x.txt", delim_whitespace=True, header=None)
df_y = pd.read_csv("../resultados/dinamica/velocidad_y.txt", delim_whitespace=True, header=None)
df_z = pd.read_csv("../resultados/dinamica/velocidad_z.txt", delim_whitespace=True, header=None)

Datos = pd.read_csv("../resultados/dinamica/gname2.txt", delim_whitespace=True, header=None, names=['t','Ec','Ep','E_tot','W','T','P_md','P_corr','F2'])
t = pd.read_csv("../resultados/dinamica/gname2.txt", delim_whitespace=True, header=None, names=['t','Ec','Ep','E_tot','W','T','P_md','P_corr','F2']).t.to_numpy()



df_x.describe()

/var/folders/15/rn_wb7ms2yq2yn7zj6rqkq880000gn/T/ipykernel_3972/327970425.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_x = pd.read_csv("../resultados/dinamica/velocidad_x.txt", delim_whitespace=True, header=None)
/var/folders/15/rn_wb7ms2yq2yn7zj6rqkq880000gn/T/ipykernel_3972/327970425.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_y = pd.read_csv("../resultados/dinamica/velocidad_y.txt", delim_whitespace=True, header=None)
/var/folders/15/rn_wb7ms2yq2yn7zj6rqkq880000gn/T/ipykernel_3972/327970425.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_z = pd.read_csv("../resultados/dinamica/velocidad_z.txt", delim_whitespace=True, header=None)
/var/folders/15/rn_wb7ms2yq2yn7zj6rqk

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.177451,0.215185,-0.379770,-0.223680,0.071201,0.138520,-0.102309,0.432582,0.322668,0.225716,...,-0.063892,-0.214347,-0.675529,-0.411383,0.689321,-0.096218,-0.005889,0.902803,-0.203755,-0.504511
std,1.224651,0.864884,1.221624,0.915862,1.074819,1.033595,1.254433,1.083769,1.173580,0.955632,...,1.111131,0.763592,1.113079,1.115042,1.488880,1.379432,0.951108,1.107911,0.734949,1.052230
min,-2.485911,-1.776635,-3.688327,-1.992191,-2.071105,-2.958209,-3.454757,-1.732572,-2.277615,-2.162899,...,-2.906703,-2.575778,-2.871570,-2.297999,-2.805866,-3.299936,-2.411101,-1.067329,-2.202838,-2.777704
25%,-0.331861,-0.161964,-1.019618,-0.931342,-0.682816,-0.353701,-0.866418,-0.267801,-0.571178,-0.474829,...,-0.699752,-0.731557,-1.505987,-1.197055,-0.156401,-0.848428,-0.722951,-0.095113,-0.665958,-1.093096
50%,0.032968,0.262808,-0.289501,-0.396377,-0.091430,0.014386,0.013543,0.349210,0.328597,0.186246,...,0.080390,-0.064136,-0.719466,-0.550488,0.489082,0.004916,-0.073721,0.746810,-0.081890,-0.631809
75%,0.916907,0.788172,0.502943,0.420825,0.903006,0.757687,0.731592,1.113384,0.953470,0.852215,...,0.786380,0.290420,0.101175,0.504720,1.669754,0.794855,0.592199,1.891324,0.203665,0.281530
max,3.372702,1.716508,2.405486,2.509175,3.105064,2.993895,3.255043,3.040846,3.391670,2.920031,...,1.714555,1.466607,1.791809,1.961992,3.930589,2.931879,2.235776,3.094723,2.380806,1.676085


### Formato de datos importados

- En las columnas tenemos las particulas de la caja etiquetadas del 0 al 499
- En las filas tenemos instantes sucesivos de tiempo grabados de nuestra simulacion.

In [14]:
df_x.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.419560,-0.431566,1.455498,-1.992191,-0.558391,-0.518082,0.042612,0.297408,-0.161098,-2.162899,...,0.039389,-0.256421,1.565880,-0.454001,0.582361,-1.329669,-0.545730,1.853191,-0.466500,-0.885440
1,0.356757,-0.506097,1.309337,-1.941383,-0.279135,-0.502747,0.012723,0.288809,-0.336422,-2.101171,...,0.053923,-0.320492,1.550292,-0.474673,0.547424,-1.361206,-0.435022,1.795817,-0.589407,-0.845959
2,0.297596,-0.544355,1.082670,-1.870929,0.002773,-0.487528,0.016633,0.291025,-0.495995,-2.020274,...,0.066436,-0.425635,1.536445,-0.497502,0.512641,-1.393405,-0.328424,1.742946,-0.692703,-0.797054
3,0.244010,-0.556245,0.777466,-1.779847,0.282411,-0.472311,0.038600,0.298928,-0.628006,-1.921314,...,0.076961,-0.552454,1.524396,-0.522581,0.478901,-1.426281,-0.231631,1.702088,-0.767358,-0.735604
4,0.195680,-0.550226,0.430688,-1.670272,0.551226,-0.457071,0.069950,0.309791,-0.727721,-1.799773,...,0.085637,-0.688657,1.514204,-0.550170,0.446961,-1.459657,-0.146998,1.675431,-0.811641,-0.654557


In [18]:
Datos.head()

,t,Ec,Ep,E_tot,W,T,P_md,P_corr,F2
0,0.01,998.4712,-1663.454,-664.9833,-897.8865,1.331295,0.366352,0.332843,643.4967
1,0.02,1012.0820,-1677.061,-664.9792,-1115.6820,1.349443,0.302827,0.269318,553.5964
2,0.03,1023.7920,-1688.775,-664.9833,-1309.8940,1.365056,0.245896,0.212388,498.7590
3,0.04,1030.8300,-1695.817,-664.9871,-1435.9910,1.374440,0.208556,0.175047,466.0179
4,0.05,1030.9110,-1695.906,-664.9953,-1459.7190,1.374548,0.200701,0.167192,480.5645


Para las distribuciones de velocidad interesan solamente la muestra de velocidades. Independientemente del tiempo. Por lo tanto aplanaremos nuestras DF en un array con todas las velocidades

In [ ]:
df_x.values.flatten() # Devuelve un array uniendo fila a fila del DF

## Preparo distribución de velocidades

In [ ]:
vx=df_x.values.flatten()
vy=df_y.values.flatten()
vz=df_z.values.flatten()

## Calculo temperaturas
En unidades reducidas $<v_x^2>=T$

In [ ]:
vx2=vx*vx
vy2=vy*vy
vz2=vz*vz

Tx=vx2.mean();#Tx_var=vx*(vx.std())
Ty=vy2.mean();#Ty_var=vy2.var()
Tz=vz2.mean();#Tz_var=vz2.var()

In [ ]:
print(f"Tx={Tx:.4f}()\nTy={Ty:.4f}()\nTz={Tz:.4f}()")

# Graficas de las distribuciones de velocidad

### Función distribución de Maxwell

In [ ]:
def Maxwell(v,T):
    return (1/np.sqrt(2*np.pi*T))*np.exp(-(v*v)/(2*T))

#x = np.linspace(vx.min(), vx.max(), 200)
#plt.plot(x,Maxwell(x,1))

In [ ]:
#from scipy.stats import norm

plt.hist(vx, bins=60, density=True, alpha=0.6, color='skyblue', label='Simulación')


x = np.linspace(vx.min(), vx.max(), 200)
y = Maxwell(x,Tx)
#plt.plot(x, norm.pdf(x, 0, np.sqrt(Tx)), 'r-', lw=2, label='Teoría (Maxwell)')
plt.plot(x,y,'r-', lw=2, label="Distribución de Maxwell")


plt.xlabel("Velocidades")
plt.ylabel("Probabilidades")
plt.title("Distribución de velocidad en eje x")
plt.grid(True)
plt.minorticks_on()
plt.legend(loc='upper right', framealpha=0.3)
plt.tight_layout()
plt.savefig("Vx.png") 
#plt.show()

In [ ]:
plt.hist(vy, bins=70, density=True, alpha=0.6, color='green', label='Simulación')

x = np.linspace(vy.min(), vy.max(), 200)
y = Maxwell(x,Ty)

plt.plot(x,y,'r-', lw=2, label="Distribución de Maxwell")


plt.xlabel("Velocidades")
plt.ylabel("Probabilidades")
plt.title("Distribución de velocidad en eje y")
plt.grid(True)
plt.minorticks_on()
plt.legend(loc='upper right', framealpha=0.3)
plt.tight_layout()
plt.savefig("Vy.png") 
#plt.show()

In [ ]:
plt.hist(vz, bins=60, density=True, alpha=0.6, color='orange', label="Distribución de velocidades")

x = np.linspace(vz.min(), vz.max(), 200)
y = Maxwell(x,Tz)

plt.plot(x,y,'r-', lw=2, label="Distribución de Maxwell")


plt.xlabel("Velocidades")
plt.ylabel("Probabilidades")
plt.title("Distribución de velocidad en eje z")
plt.grid(True)
plt.minorticks_on()
plt.legend(loc='upper right', framealpha=0.3)
plt.tight_layout()
plt.savefig("Vz.png") 
#plt.show()

# La función H de Boltzman

$$H(t) = \int f(\mathbf{v},t)\,\ln f(\mathbf{v},t)\,d\mathbf{v}$$ con $\int f(\mathbf{v},t)\,d\mathbf{v} = 1$

### El teorema H de Boltzman
El teorema H de Boltzman establece que $\frac{dH}{dt} \le 0$  y que H alcanza un mínimo constante cuando el sistema llega al equilibrio, momento en el cual $f(\mathbf{v})$ tiene la forma de Maxwell–Boltzmann.


In [ ]:
df_x

index=np.arange(0,len(df_x)) // 50

if "tiempo" not in  df_x.columns:
    df_x.insert(0, "tiempo", t)
    
df_x["index"]=index
H=df_x.set_index("index").groupby("index").apply(lambda df: pd.Series({"tiempo_medio":df.tiempo.mean(),"Distribucion":df.iloc[:,1:].to_numpy().flatten()}))


In [ ]:
f, edges = np.histogram(vx, bins=100, density=True)
v_mid = 0.5*(edges[1:] + edges[:-1]) 


#Tengo que hacer varios f(v) por ejemplo agrupando los vx cada 200 y asignandoles el valor de su tiempo medio 
# La grafica no tiene que ser perfecta, solo hay que ver que deja de caer la H
# Entonces cada 200 grabaciones hago un f(v) y le asigno t=t_medio
# Integro los f(v) y saco una H para cada 200 pasos.
# Hago grafica de H

def simpson_discreto(x, y):
    n = len(x)
    if n % 2 == 0:
        n -= 1  # asegurar impar para Simpson
        x, y = x[:n], y[:n]
    h = (x[-1]-x[0])/(n-1)
    S = y[0] + y[-1] + 4*np.sum(y[1:-1:2]) + 2*np.sum(y[2:-2:2])
    return h*S/3

def H_boltzmann(v):
    # construimos histograma normalizado f(v)
    f, bins = np.histogram(v, bins=100, density=True)
    v_medios = 0.5 * (bins[1:] + bins[:-1])
    # evitamos log(0)
    f_nonzero = f[f > 0]
    v_nonzero = v_medios[f > 0]
    # calculamos H = ∫ f ln f dv
    return simpson_discreto(v_nonzero, f_nonzero * np.log(f_nonzero))

H["H_Boltzman"]=H.Distribucion.map(H_boltzmann)



H


In [ ]:
plt.plot(H.tiempo_medio,H.H_Boltzman, "r-")

print(f"<H>={H.H_Boltzman.mean():.4f} ± {H.H_Boltzman.std():.4f}")
plt.xlabel("Tiempo")
plt.ylabel("H de Boltzmann")
plt.title("H de Boltzmann")
plt.grid(True)
plt.minorticks_on()
#plt.legend(loc='best', framealpha=0.3)
plt.tight_layout()

plt.savefig("H_Boltzman.png") 


# Sobre la energía y la presión

In [ ]:
df=pd.read_csv("gname2.txt", delim_whitespace=True, header=None, names=["t","Ep","Ec","Et"])

E_total=df.Et
Ec=df.Ec
Ep=df.Ep


plt.plot(t,Ep,"r-", label="Energía potencial")
plt.plot(t,Ec,"b-", label="Energía cinética")
plt.plot(t,E_total,"g-", label="Energía total")

plt.xlabel("Tiempo")
plt.ylabel("Energía")
plt.title("Energías")
plt.grid(True)
plt.minorticks_on()
plt.legend(loc='best', framealpha=0.3)
plt.tight_layout()
plt.savefig("Energias.png")

In [ ]:
plt.plot(t,E_total,"g-", label="Energía total")
plt.xlabel("Tiempo")
plt.ylabel("Energía")
plt.title("Energías")
plt.grid(True)
plt.minorticks_on()
plt.legend(loc='best', framealpha=0.3)
plt.tight_layout()
plt.savefig("Energia_total.png")



In [ ]:
print(E_total.agg(["mean","std"]))